In [3]:
import logging
logging.basicConfig(
  level=logging.INFO,  
  format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",  
)
logging.getLogger("httpx").setLevel(logging.WARNING)
LOGGER = logging.getLogger(__name__)

import os
import tempfile
from bondable.bond.providers.provider import Provider
from bondable.bond.providers.agent import Agent
from bondable.bond.broker import Broker, BrokerConnectionEmpty
from bondable.bond.definition import AgentDefinition
from bondable.bond.functions import Functions
from bondable.bond.config import Config
from IPython.display import Image, display
from dotenv import load_dotenv
from typing import List
import pandas as pd
import base64
import threading


load_dotenv(override=True)

user_id = "jmcarny@gmail.com"


config = Config.config()
provider: Provider = config.get_provider()
builtin_functions = Functions.functions()
provider.cleanup(user_id=user_id)

def get_first_agent_id(agent_name):
    agents: List[Agent] = provider.agents.get_agents_by_name(agent_name)
    if len(agents) > 0:
        return agents[0].get_agent_id()
    return None

def display_message (message):
    if message.role == 'system':
        LOGGER.debug(f"Received system message, ignoring {message.message_id}")
        return
    if message.type == "text":
        print(f"[{message.message_id}/{message.role}] => {message.clob.get_content()}")
    elif message.type == "image_file":
        print(f"[{message.message_id}/{message.role}] => ")
        content = message.clob.get_content()
        if content.startswith('data:image/png;base64,'):
            base64_image = content[len('data:image/png;base64,'):]
            image_data = base64.b64decode(base64_image)
            display(Image(data=image_data))
        else:
            print(content)
    else:
        LOGGER.error(f"Unknown message type {type}")

def print_responses (user_id, prompts, agent_def: AgentDefinition):
    # Create_thread now returns an ORM object, we need its ID.
    agent = provider.agents.create_or_update_agent(agent_def=agent_def, user_id=user_id)
    created_thread = provider.threads.create_thread(user_id=user_id, name=f"PrintResponses for {agent.get_name()}")
    thread_id = created_thread.thread_id
    try:
        broker = Broker.broker()
        conn = broker.connect(thread_id=thread_id, subscriber_id=user_id)
        for prompt in prompts:
        
            message = agent.create_user_message(prompt, thread_id)
            thread = threading.Thread(target=agent.broadcast_response, args=(None, thread_id), daemon=True)
            thread.start()
            while True:
                try:
                    bond_msg = conn.wait_for_message(timeout=5)
                    if bond_msg is None:
                        break
                    display_message(bond_msg)
                    if bond_msg.is_done:
                        break
                except BrokerConnectionEmpty:
                    continue
                except Exception as e:
                    LOGGER.error(f"Error: {e}")
                    break
            thread.join()

        conn.close()
    finally:
        provider.threads.delete_thread(thread_id=thread_id, user_id=user_id)


2025-05-27 16:41:04,706 - bondable.bond.providers.provider - INFO - Cleaning up resources for user_id: jmcarny@gmail.com
2025-05-27 16:41:04,706 - bondable.bond.providers.agent - INFO - Cleaning up resources for user_id: jmcarny@gmail.com
2025-05-27 16:41:05,769 - bondable.bond.providers.openai.OAIAAgent - INFO - Successfully deleted agent asst_C32tXZF0pRaotWRNx8aqzkgL from provider.
2025-05-27 16:41:05,772 - bondable.bond.providers.agent - INFO - Deleted 1 local DB records for agent_id: asst_C32tXZF0pRaotWRNx8aqzkgL
2025-05-27 16:41:05,773 - bondable.bond.providers.agent - INFO - Deleted agent with agent_id: asst_C32tXZF0pRaotWRNx8aqzkgL - Success: True


In [ ]:
agent_def = AgentDefinition(
    name="Simple Agent",
    id=get_first_agent_id("Simple Agent"),
    description="Pirate Agent.",
    instructions="Answer requests from user like a pirate.",
    model="gpt-4o-nano",
    metadata={'visible': 'True'},
    tools=[{"type": "code_interpreter"}],
    user_id=user_id
)
print_responses(user_id=user_id, prompts=['tell me a joke'], agent_def=agent_def)

2025-05-27 16:40:42,124 - bondable.bond.providers.openai.OAIAAgent - INFO - No ID provided. Attempting to create new agent named 'Simple Agent'
2025-05-27 16:40:42,124 - bondable.bond.providers.openai.OAIAAgent - INFO - Creating new OpenAI assistant: Simple Agent
2025-05-27 16:40:42,574 - bondable.bond.providers.openai.OAIAAgent - INFO - Successfully created new agent 'Simple Agent' with ID 'asst_C32tXZF0pRaotWRNx8aqzkgL'.
2025-05-27 16:40:42,574 - bondable.bond.broker - INFO - Created Broker instance
2025-05-27 16:40:42,576 - bondable.bond.providers.agent - INFO - Creating new agent record for agent_id: asst_C32tXZF0pRaotWRNx8aqzkgL
2025-05-27 16:40:43,343 - bondable.bond.providers.openai.OAIAThreads - INFO - Successfully created thread thread_8qKEUxxyLRQqDTILcbgXUlbs from provider


[msg_3wfhC0bmQNZRMCEr7AehYAlz_0/assistant] => Arrr, why did the pirate go to school? Because he wanted to improve his arrrticulation! Har har har!


2025-05-27 16:40:52,767 - bondable.bond.providers.openai.OAIAThreads - INFO - Successfully deleted thread thread_8qKEUxxyLRQqDTILcbgXUlbs from openai
2025-05-27 16:40:52,768 - bondable.bond.providers.threads - INFO - Deleted 1 DB records for thread_id: thread_8qKEUxxyLRQqDTILcbgXUlbs and user_id: jmcarny@gmail.com - remaining users: 0


In [ ]:
agent_def = AgentDefinition(
    name="Function Agent",
    id=get_first_agent_id("Function Agent"),
    description="Say hello to the user.",
    instructions="Call the simple method 'hello' with the name provided by the user. If no name is provided, ask the user to provide a name.",
    metadata={'visible': 'True'},
    tools=[builtin_functions.hello],
    model="gpt-4o-nano",
    user_id=user_id
)
print_responses(user_id=user_id, prompts=['my name is john'], agent_def=agent_def)

2025-05-27 16:42:34,515 - bondable.bond.providers.openai.OAIAAgent - INFO - No ID provided. Attempting to create new agent named 'Function Agent'
2025-05-27 16:42:34,515 - bondable.bond.providers.openai.OAIAAgent - INFO - Creating new OpenAI assistant: Function Agent
2025-05-27 16:42:35,265 - bondable.bond.providers.openai.OAIAAgent - INFO - Successfully created new agent 'Function Agent' with ID 'asst_MSSXQSToDhkn1MKG3JEQpS53'.
2025-05-27 16:42:35,266 - bondable.bond.providers.agent - INFO - Creating new agent record for agent_id: asst_MSSXQSToDhkn1MKG3JEQpS53
2025-05-27 16:42:35,572 - bondable.bond.providers.openai.OAIAThreads - INFO - Successfully created thread thread_GiE944nbkmSjsUK7dbhDD8lW from provider
2025-05-27 16:42:39,978 - bondable.bond.functions - INFO - Saying hello to: john


[msg_klor10h8KnHpofcYiJ6u2Z8A_0/assistant] => Hello, john!


2025-05-27 16:42:50,422 - bondable.bond.providers.openai.OAIAThreads - INFO - Successfully deleted thread thread_GiE944nbkmSjsUK7dbhDD8lW from openai
2025-05-27 16:42:50,423 - bondable.bond.providers.threads - INFO - Deleted 1 DB records for thread_id: thread_GiE944nbkmSjsUK7dbhDD8lW and user_id: jmcarny@gmail.com - remaining users: 0


In [ ]:
agent_def = AgentDefinition(
    name="Synth Agent",
    id=get_first_agent_id("Synth Agent"),
    description="An agent that will load synthetic data about people and answer questions",
    instructions="""When you begin you should create a synthetic data set that has the weight 
    and height of 100 people. You should answer questions about the data set
    """,
    tools=[{"type": "code_interpreter"}],
    metadata={'visible': 'True', 'initial_prompt': 'Generate the data set'},
    model="gpt-4o-nano",
    user_id=user_id
)
agent = provider.agents.create_or_update_agent(agent_def=agent_def, user_id=user_id)
prompts = [
  agent.get_metadata_value('initial_prompt'),
  "How many people are there",
  "What is the average height",
  # "Show me a scatterplot of height vs weight"
]
print_responses(user_id, prompts, agent_def)

In [ ]:
# Create a sample data of users
data = {
  'Customer_ID': [1, 2, 3],
  'First_Name': ['Jack', 'Jane', 'Doe'],
  'Last_Name': ['Doe', 'Smith', 'Johnson'],
  'Email': ['john.doe@example.com', 'jane.smith@example.com', 'doe.johnson@example.com'],
  'Phone_Number': ['123-456-7890', '234-567-8901', '345-678-9012'],
  'Region': ['North', 'South', 'East'],
  'Salesperson': ['Alice', 'Bob', 'Charlie'],
  'Address': ['123 Elm St', '456 Oak St', '789 Pine St'],
  'ZIP_Code': ['12345', '23456', '34567'],
  'Last_Purchase_Amount': [100.50, 200.75, 150.00]
}


df = pd.DataFrame(data)
data_file = tempfile.NamedTemporaryFile(suffix=".csv", delete=True)
df.to_csv(data_file.name, index=False)

initial_prompt = "What are all of the customer names?"
agent_def = AgentDefinition(
    name="File Agent",
    id=get_first_agent_id("File Agent"),
    description="An agent that will load data about people and answer questions",
    instructions="""
    Answer questions about user data in the attached CSV file. The first row contains the
    names of the columns. 
    """,
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "files": [(data_file.name, None)]
      }
    },
    metadata={'visible': 'True', 'initial_prompt': initial_prompt},
)
print_responses(user_id, [initial_prompt], agent_def)
os.remove(data_file.name)


In [ ]:
# Create a sample data of users
data = {
  'Customer_ID': [1, 2, 3],
  'First_Name': ['Jack', 'Jane', 'Doe'],
  'Last_Name': ['Doe', 'Smith', 'Johnson'],
  'Email': ['john.doe@example.com', 'jane.smith@example.com', 'doe.johnson@example.com'],
  'Phone_Number': ['123-456-7890', '234-567-8901', '345-678-9012'],
  'Region': ['North', 'South', 'East'],
  'Salesperson': ['Alice', 'Bob', 'Charlie'],
  'Address': ['123 Elm St', '456 Oak St', '789 Pine St'],
  'ZIP_Code': ['12345', '23456', '34567'],
  'Last_Purchase_Amount': [100.50, 200.75, 150.00]
}

df = pd.DataFrame(data)
html_file = tempfile.NamedTemporaryFile(suffix=".html", delete=True)
df.to_html(html_file.name, index=False)

initial_prompt = "What are all of the customer names?"
agent_def = AgentDefinition(
    name="HTML File Agent",
    id=get_first_agent_id("HTML File Agent"),
    description="An agent that will load data about people and answer questions",
    instructions="""
    Answer questions about user data in the attached HTML file.
    """,
    tools=[{"type": "file_search"}],
    tool_resources={
      "file_search": {
        "files": [(html_file.name, None)]
      }
    },
    metadata={'visible': 'True', 'initial_prompt': initial_prompt},
)
print_responses(user_id, [initial_prompt], agent_def)
os.remove(html_file.name)
